# The sociology of rats

In a population of interacting individuals, for instance **_rats_** in a cage, the mood of one individual can influence the others. Moreover, a collective behaviour can emerge if the interactions are sufficiently numerous, for instance if the density of the rats gets sufficiently large.

A simple model for studying this phenomenon considers a population of individuals (rats!) which can be either _calm_ or _nervous_.

The rats leave in a two-dimensional grid and each rat moves randomly from site to site.

At each time step a rat moves to a nearest-neighbour site, chosen randomly.
Two rats cannot occupy the same site, hence if no free site is available nearby, the rat does not move.

At each step the rat checks the state of the neighbouring rats in the neighbouring site: if they are all calm, the rat becomes calm. If any is nervous, the rat gets nervous as well. Moreover, with a small probability $p$, at each step a rat can become nervous spontaneously.

It can be proven that the fraction of nervous rats in the population undergoes a _phase transition_ driven by the density $\rho$ of rats in the lattice: below a certain _critical value_ $\rho_c$ the fraction of nervous rats is small (in fact it would be zero on an infinitely large lattice). Raising the density of rats above $\rho_c$, the fraction of nervous rats increases abruptly.

## Assignment 1

Define a _class_ which describes rats living at the nodes of a square $L\times L$ lattice, on which we will assume **periodic** boundary conditions.

Each node or site of the lattice can be in three possible states:
* empty;
* occupied by a calm rat;
* occupied by a nervous rat;

you are free to choose how to represent these three states.

To be useful, the class will require several methods:
* the _constructor_ method will accept the size of the lattice, the density $\rho$ of rats to be placed randomly on it, the probability $p$ of a rat getting nervous.
* the _update_ method will allow generating a new configuration by
    * randomly change calm rats into nervous ones with probability $p$;
    * move each rat to a neighbouring free site
    * check the status of nearest neighbour rats: if all are calm, the rat becomes calm as well. If any is nervous, the rat gets nervous as well.
* the _measure_ method will count the number of nervous rats, returning the fraction of nervous rats over the total population.
* the _evolve_ method will accept as parameters the number $N_t$ of _thermalisation_ steps, the number $N_m$ of _measurements_ and the number $N_d$ of _drop_ steps. It will evolve the system first by $N_t$ time steps to thermalise it, then it will evolve the system for $N_m\times N_d$ steps and take one measurement every $N_d$ steps. It will return the fraction $f_n$ of nervous rats averaged over the simulation, its standard deviation $\sigma_n$, and also the vector of the $N_m$ individual measurements taken.
* the _draw_ method will show a plot of the status of the lattice, in which rats are drawn as dots on the lattice, with different colours depending on their status.

**Note 1**: this class can be written in various ways, equivalent from the point of view of functionality, but possibly with very different memory footprint and computational efficiency.
For instance, one may choose to allocate the whole lattice; or one can allocate only the rats, namely their state and coordinates on the lattice, which can be more efficient when the density of rats is small.

The functions of the `numpy` library that we have exploited during the course to parallelize the code, for instance `numpy.where` which allows to choose elements on the basis of a condition, can be of great help here.

**Note 2**: beware, two rats cannot be on the same site. It is necessary to take this into account when moving the rats.

**Note 3**: if you have doubts about what it means to take measurements, computing averages and standard deviations, please review the lesson [06.03](https://blended.uniurb.it/moodle/mod/folder/view.php?id=458258) about the Ising model.

To check the code, try running the _evolve_ method with $p=0.01$, $L=30$, using $N_t=1000, N_m=1000, N_d=10$ to compute the average fractions $f_n$ of nervous rats for different values of $\rho$. For instance choose $\rho = 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5$ and see what happens. By plotting the measured averaged $f_n$ as a function of $\rho$, you should observe small values of $f_n$ for small values of $\rho$, then a transition to larger values as $\rho$ increases, and you should be able to _roughly_ locate the critical value $\rho_c$.

## Assignment 2

Keep $p=0.01$ but choose now a larger lattice, at least $L=50$ if your computer manages to complete the simulation within reasonable time limits.
Otherwise, choose a lattice as large as you find viable.

Perform simulations with $N_t=1000, N_m=1000, N_d=10$ at several values of the density $\rho$ of rats, in order to better visualize the transition occurring at $\rho_c$, by plotting the fraction $f_n$ of nervous rats (averaged over the simulation) against the density $\rho$.
You are free to choose the number of $\rho$ values and their spacing, in order to obtain a clearer plot; your goal is to locate more accurately the value of $\rho_c$.

**Note:** since the goal is to locate the value of $\rho_c$, and considering that larger $\rho$ values typically require longer simulations (more rats!), it can be convenient to limit the simulation to values close to the $\rho_c$ value estimated on smaller lattices.

Draw also a plot of the standard deviation $\sigma_n$, as a function of $\rho$; it should display larger values when being close to $\rho_c$.

# Assignment 3

Set the lattice size at $L=50$, the probability $p=0.01$, and choose three values of $\rho$
* one well below $\rho_c$
* one close to $\rho_c$
* one well above $\rho_c$

For each chosen value of $\rho$ perform a simulation with $N_t=1000, N_m=10000, N_d=1$, obtaining the individual measurements of $f_n$.

Compute the _autocorrelation_ of the values of $f_n$, and see if there are differences in its shape between the three cases.

**Note** If you have doubts about how to compute an autocorrelation, please review lesson [06.02](https://blended.uniurb.it/moodle/mod/folder/view.php?id=458258) about Distributions.

## Assignment 4

Add an *animation* method to the class which allows to create an animation of the evolution of the lattice.

Please exploit the techniques of the `matplotlib` that we have introduced in various lessons, for instance in [04.06](https://blended.uniurb.it/moodle/pluginfile.php/792656/mod_folder/content/0/04_06_Reaction_Diffusion_Solved.ipynb?forcedownload=1)

Try producing a few animations corresponding to the parameters used for Assignment 3.

In [ ]:
from enum import Enum

class rat_state(Enum):
    CALM = 1
    ANGRY = 2

In [ ]:
class Rat:

    def __init__(self, **kwargs):
        self.rat_state = rat_state(kwargs.get('State', rat_state.CALM ))
    
    def __str__(self) -> str:
        return 'Rat is {}'.format(self.rat_state)
    

    